# 현재 작업 중입니다.

<!-- # [Module 1.6] Horovod 훈련


본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 아래는 세이지메이커의 어떤 피쳐도 사용하지 않고, PyTorch 만을 사용해서 훈련 합니다. -->

# PyTorch CIFAR-10 local training  



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-cnn-cifar10"

role = sagemaker.get_execution_role()

In [3]:
import os
import subprocess

instance_type = "local"

try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
except:
    pass

print("Instance type = " + instance_type)

Instance type = local_gpu


### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [4]:
inputs = sagemaker_session.upload_data(path="../data", bucket=bucket, key_prefix="data/cifar10")
print("s3 inputs: ", inputs)

s3 inputs:  s3://sagemaker-us-east-1-057716757052/data/cifar10


# Construct a script for training 
- epoch 10 , 20
    - 각각 테스트 정확도 55.2, 62.9


In [5]:
import os
import subprocess

instance_type = "local_gpu"
# instance_type = "ml.p3.8xlarge"

job_name ='cifar10-horovod'

## 시스템의 이전 도커 컨테이너 삭제
- 아래와 같은 명령어를 사용하여 저장 공간을 확보 합니다.

### 도커 컨테이너 모두 삭제

In [7]:
! df -h
! docker container prune -f --all
! df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs         30G   76K   30G   1% /dev
tmpfs            30G  320K   30G   1% /dev/shm
/dev/xvda1      104G   91G   14G  88% /
/dev/xvdf       984G  8.6G  925G   1% /home/ec2-user/SageMaker
unknown flag: --all
See 'docker container prune --help'.
Filesystem      Size  Used Avail Use% Mounted on
devtmpfs         30G   76K   30G   1% /dev
tmpfs            30G  320K   30G   1% /dev/shm
/dev/xvda1      104G   91G   14G  88% /
/dev/xvdf       984G  8.6G  925G   1% /home/ec2-user/SageMaker


### 도커 이미지 모두 삭제

In [8]:
! df -h
! docker image prune -f --all
! df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs         30G   76K   30G   1% /dev
tmpfs            30G  320K   30G   1% /dev/shm
/dev/xvda1      104G   91G   14G  88% /
/dev/xvdf       984G  8.6G  925G   1% /home/ec2-user/SageMaker
Total reclaimed space: 0B
Filesystem      Size  Used Avail Use% Mounted on
devtmpfs         30G   76K   30G   1% /dev
tmpfs            30G  320K   30G   1% /dev/shm
/dev/xvda1      104G   91G   14G  88% /
/dev/xvdf       984G  8.6G  925G   1% /home/ec2-user/SageMaker


In [12]:
from sagemaker.pytorch import PyTorch

cifar10_estimator = PyTorch(
    entry_point="train_horovod.py",    
    source_dir='source',    
    base_job_name = job_name,
    role=role,
    framework_version='1.6.0',
    py_version='py3',
    train_instance_count=1,
    train_instance_type=instance_type,
    hyperparameters={"epochs": 3, 
                     'lr': 0.001,
                     'batch-size': 64,
                     'log-interval' : 100,
                     "backend": "gloo",                     
                    },    
)
cifar10_estimator.fit({"training" : inputs})

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Creating bepciuprgg-algo-1-5icaf ... 
Creating bepciuprgg-algo-1-5icaf ... done
Attaching to bepciuprgg-algo-1-5icaf
bepciuprgg-algo-1-5icaf | 2021-09-27 07:28:33,416 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
bepciuprgg-algo-1-5icaf | 2021-09-27 07:28:33,441 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bepciuprgg-algo-1-5icaf | 2021-09-27 07:28:33,445 sagemaker_pytorch_container.training INFO     Invoking user training script.
bepciuprgg-algo-1-5icaf | 2021-09-27 07:28:33,609 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
bepciuprgg-algo-1-5icaf | /opt/conda/bin/python3.6 -m pip install -r requirements.txt
bepciuprgg-algo-1-5icaf | Requirement already satisfied: torch==1.6.0 in /opt/conda/lib/python3.6/site-packages (from -r requirements.txt (line 1)) (1.6.0)
bepciuprgg-algo-1-5icaf | Requirement already satisfied: torchvision==0.7.0 in /opt/conda/lib/python3

#### 로컬모드에서 도커 이미지 다운로드 된 것을 확인

In [11]:
! docker image ls

REPOSITORY                                                      TAG                 IMAGE ID            CREATED             SIZE
763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training   1.6.0-gpu-py3       30e42e4701a4        5 months ago        8.6GB


In [ ]:
from sagemaker.pytorch import PyTorch

instance_type

cifar10_estimator = PyTorch(
    entry_point="train_horovod.py",    
    source_dir='source',    
    base_job_name = job_name,
    role=role,
    framework_version='1.6.0',
    py_version='py3',
    train_instance_count=1,
    train_instance_type=instance_type,
    hyperparameters={"epochs": 3, 
                     'lr': 0.001,
                     'batch-size': 64,
                     "backend": "gloo",                     
                    },    
)
cifar10_estimator.fit({"training" : inputs})

In [ ]:
horovod_artifact_path = cifar10_estimator.model_data
print("horovod_artifact_path: ", horovod_artifact_path)


%store horovod_artifact_path

In [ ]:
! aws s3 ls {horovod_artifact_path} --recursive